In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import ParameterGrid
from sklearn.pipeline import Pipeline
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix, hstack
import itertools
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing
from sklearn.metrics import coverage_error
from sklearn.svm import SVC
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.neural_network import MLPClassifier
import scipy.sparse
from sklearn.cluster import KMeans

In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from bs4 import BeautifulSoup

#from sklearn.preprocessing import StandardScaler

from scipy.sparse import coo_matrix, hstack


In [3]:
#Dataset loading
dataset = pd.read_csv('Cleaned_Posts.csv', nrows = None, index_col=0)
dataset.shape

(49399, 10)

In [4]:
#Any np.nan ?
dataset.isnull().sum()

TText             0
TText_NEG         0
PText             0
TCode         10054
PCode          9627
TTitle            0
TTitle_NEG        0
Title             0
PTags             0
Tags              0
dtype: int64

In [5]:
#np.nan cleaning
dataset['TCode'] = dataset['TCode'].fillna('None')
dataset['PCode'] = dataset['PCode'].fillna('None')

In [6]:
#Any np.nan ?
dataset.isnull().sum()

TText         0
TText_NEG     0
PText         0
TCode         0
PCode         0
TTitle        0
TTitle_NEG    0
Title         0
PTags         0
Tags          0
dtype: int64

In [7]:
#Tags cleaning
dataset['PTags'] = dataset['Tags'].apply(lambda x: [tag.name for tag in BeautifulSoup(x, 'html.parser').find_all()])

In [8]:
df = pd.DataFrame(dataset['PTags'].tolist()).stack().value_counts()
#df = df[df>50]
df.shape

(13792,)

In [9]:
existintags = set(df.index)
existintags

{'fastexport',
 'newtons-method',
 'rcu',
 'xquartz',
 'jjs',
 'ssh.net',
 'quosure',
 'form-api',
 'ember-router',
 'tps',
 'font-awesome',
 'mapbox',
 'wpf-controls',
 'file-system-storage',
 'simplexmlrpcserver',
 'android-layout-editor',
 'plesk',
 'yeoman',
 'facebook-pixel',
 'hierarchy',
 'android-toast',
 'rabbitmqctl',
 'chez-scheme',
 'gulp-uglify',
 'aws-vpc',
 'flags',
 'srcset',
 'webviewclient',
 'data-visualization',
 'alibaba-cloud-ecs',
 'spreadsheetgear',
 'cfml',
 'opentk',
 'indexoutofboundsexception',
 'dma',
 'r-caret',
 'ms-release-management',
 'urho3d',
 'reactstrap',
 'execution',
 'winscp',
 'jsse',
 'mumin',
 'brackets',
 'formulas',
 'ninject-interception',
 'internet-radio',
 'mat-autocomplete',
 'codec',
 'cpanel',
 'uibezierpath',
 'scala',
 'ssrs-2012',
 'depth',
 'unet',
 'code-map',
 'git-revert',
 'isolation-level',
 'ag',
 'specrun',
 'weld',
 'sqlcmd',
 'implicit-conversion',
 'nouislider',
 'reader',
 'd',
 'row-major-order',
 'itk',
 'preloadjs',

In [124]:
#Downsampling
dataset = dataset.sample(5000)
dataset.shape

(5000, 10)

In [10]:

class CustomLDA(BaseEstimator, TransformerMixin):

    def __init__(self, **params):        
        self.vect_text_params = {'ngram_range':(1,1), 'max_df':1., 'min_df':1, 'max_features':None}
        self.vect_title_params = {'ngram_range':(1,1), 'max_df':1., 'min_df':1, 'max_features':None}
        self.vect_code_params = {'ngram_range':(1,1), 'max_df':1., 'min_df':1, 'max_features':None}
        
        self.lda_params = {'n_components':10, 'n_jobs':2, 'random_state':0, 'doc_topic_prior':None, 'topic_word_prior':None}
        
        self.clf_params = {'ntopwords':5, 'ntopics':5}
        
        self.params = {**{'vect_text_'+k:v for k,v in self.vect_text_params.items()},
                       **{'vect_title_'+k:v for k,v in self.vect_title_params.items()},
                       **{'vect_code_'+k:v for k,v in self.vect_code_params.items()},
                       **{'lda_'+k:v for k,v in self.lda_params.items()},
                       **{'clf_'+k:v for k,v in self.clf_params.items()},
                       **params}
        
        self.update_params()

    def predict(self, X, y=None):
        ntopwords = self.clf_params['ntopwords']
        ntopics = self.clf_params['ntopics']
        W = self.transform(X)
        components = self.components_()
        feature_names = self.get_feature_names()

        toreturn = []
        
        for document in W:
            #select the most important topics
            docprediction = []
            for topic in document.argsort()[:-ntopics-1:-1]:
                tags = [feature_names[i] for i in components[topic].argsort()[:-ntopwords-1:-1]]
                docprediction.append(tags)
            toreturn.append(docprediction)
        return toreturn
    
    def fit(self, X, y=None):
        dftext = X['TText']
        dftitle = X['TTitle']
        dfcode = X['TCode']
        #Text preparation
        self.textcvect = CountVectorizer(tokenizer=None, vocabulary=None, **self.vect_text_params)
        text = self.textcvect.fit_transform(dftext)
        
        #Title preparation
        self.titlecvect = CountVectorizer(tokenizer=None, vocabulary=None, **self.vect_title_params)
        title = self.titlecvect.fit_transform(dftitle)
        
        #Code preparation
        self.codecvect = CountVectorizer(tokenizer=None, vocabulary=None, **self.vect_code_params)
        code = self.codecvect.fit_transform(dfcode)
        
        #LDA preparation
        self.lda = LatentDirichletAllocation(learning_method = 'batch', **self.lda_params)
        complete = scipy.sparse.hstack((text, title, code))
        self.lda.fit(complete)
        
        return self
    
    def transformcvect(self, X, y=None):
        dftext = X['TText']
        dftitle = X['TTitle']
        dfcode = X['TCode']
        
        #Text preparation
        textvect = self.textcvect.transform(dftext)
        
        #Title preparation
        titlevect = self.titlecvect.transform(dftitle)
    
        #Code preparation
        codevect = self.codecvect.transform(dfcode)
        
        return scipy.sparse.hstack((textvect, titlevect, codevect))
        
    def transform(self, X, y=None):
        
        lda = self.lda.transform(self.transformcvect(X))
        return lda
    
    def get_feature_names(self):
        textfnames = self.textcvect.get_feature_names()
        titlefnames = self.titlecvect.get_feature_names()
        codefnames = self.codecvect.get_feature_names()
        toreturn = []
        toreturn.extend(textfnames)
        toreturn.extend(titlefnames)
        toreturn.extend(codefnames)
        
        return toreturn
    
    def components_(self):        
        return self.lda.components_
    
    def get_params(self, deep=True):
        return self.params
    
    def set_params(self, **params):
        self.params = {**self.params, **params}
        self.update_params()
        return self
        
    def update_params(self):
        
        self.vect_text_params = {k[10:]:v for k,v in self.params.items() if k.startswith('vect_text_')}
        self.vect_title_params = {k[11:]:v for k,v in self.params.items() if k.startswith('vect_title_')}
        self.vect_code_params = {k[10:]:v for k,v in self.params.items() if k.startswith('vect_code_')}
        
        self.lda_params = {k[4:]:v for k,v in self.params.items() if k.startswith('lda_')}
        
        self.clf_params = {k[4:]:v for k,v in self.params.items() if k.startswith('clf_')}

        return self

    def score(self, X=None, y=None):
        numcomponent = self.lda_params['n_components']
        self.nametopics()
        topicsscore = [1/len(x) if len(x) !=0 else 0 for x in self.topicsnamed.values()]
        return numcomponent * np.array(topicsscore).mean()
    
    def perplexity(self, X, sub_sampling=False):
        return self.lda.perplexity(self.transformcvect(X))
    
    def nametopics(self):
        self.topicsnamed = {}
        self.namedtopics = {}
        ntopwords = self.clf_params['ntopwords']
        ntopics = self.clf_params['ntopics']
        components = self.components_()
        feature_names = self.get_feature_names()
        
        for topicnum, topiccomposition in enumerate(components):
            tags = [feature_names[i] for i in topiccomposition.argsort()[:-ntopwords-1:-1]]
            self.topicsnamed[topicnum] = [tag for tag in tags if tag in existintags]
            for tag in tags:
                if tag in existintags:
                    if tag in self.namedtopics.keys():
                        temp = list(self.namedtopics[tag])
                        temp.append(topicnum)
                        self.namedtopics[tag] = set(temp)
                    else:
                        self.namedtopics[tag] = set([topicnum])
        
        return self


customLDA = CustomLDA()


Some docs about the parameters : https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/

In [11]:
params = {'clf_ntopics': 3,
 'clf_ntopwords': 3,
 'lda_n_components': 50,
 'lda_n_jobs': 2,
 'lda_random_state': 0,
 'vect_code_max_df': 1.0,
 'vect_code_max_features': 15000,
 'vect_code_min_df': 1,
 'vect_code_ngram_range': (1, 3),
 'vect_text_max_df': 1.0,
 'vect_text_max_features': 20000,
 'vect_text_min_df': 1,
 'vect_text_ngram_range': (1, 3),
 'vect_title_max_df': 1.0,
 'vect_title_max_features': 70000,
 'vect_title_min_df': 1,
 'vect_title_ngram_range': (1, 2)}

customLDA = CustomLDA(**params)
customLDA.fit(dataset[['TText', 'TTitle', 'TCode']])

CustomLDA(clf_ntopics=3, clf_ntopwords=3, lda_doc_topic_prior=None,
     lda_n_components=50, lda_n_jobs=2, lda_random_state=0,
     lda_topic_word_prior=None, vect_code_max_df=1.0,
     vect_code_max_features=15000, vect_code_min_df=1,
     vect_code_ngram_range=(1, 3), vect_text_max_df=1.0,
     vect_text_max_features=20000, vect_text_min_df=1,
     vect_text_ngram_range=(1, 3), vect_title_max_df=1.0,
     vect_title_max_features=70000, vect_title_min_df=1,
     vect_title_ngram_range=(1, 2))

In [12]:
#[1/len(x) if len(x) !=0 else 0 for x in customLDA.topicsnamed.values()]
customLDA.score()

32.666666666666671

In [25]:
#gridsearch to optimize the parameters

topic = 100

params = {'clf_ntopics': 3,
 'clf_ntopwords': 3,
 'lda_n_components': topic,
 'lda_n_jobs': 2,
 'lda_random_state': 0,
 'vect_code_max_df': 0.8,
 'vect_code_max_features': 15000,
 'vect_code_min_df': 5,
 'vect_code_ngram_range': (1, 2),
 'vect_text_max_df': 0.8,
 'vect_text_max_features': 15000,
 'vect_text_min_df': 10,
 'vect_text_ngram_range': (1, 3),
 'vect_title_max_df': 0.8,
 'vect_title_max_features': 15000,
 'vect_title_min_df': 5,
 'vect_title_ngram_range': (1, 2)}

customLDA = CustomLDA(**params)

#parameters = {'vect_title_min_df': [5], 'vect_title_max_df': [0.8,], 'vect_title_max_features': [15000, 20000], 'vect_code_ngram_range': [(1,2), (1,3)],}
parameters = {'lda_doc_topic_prior':[ 1/(topic*1.2), 1/(topic), 1/(topic*0.8)], 'lda_topic_word_prior':[ 1/(topic)]}


clf = GridSearchCV(customLDA, parameters, cv=2, return_train_score=True, refit=True, verbose = 3)
clf.fit(dataset[['TText', 'TTitle', 'TCode']])

Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV] lda_doc_topic_prior=0.008333333333333333, lda_topic_word_prior=0.01 
[CV]  lda_doc_topic_prior=0.008333333333333333, lda_topic_word_prior=0.01, score=55.00000000000001, total= 9.6min
[CV] lda_doc_topic_prior=0.008333333333333333, lda_topic_word_prior=0.01 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  9.6min remaining:    0.0s


[CV]  lda_doc_topic_prior=0.008333333333333333, lda_topic_word_prior=0.01, score=59.83333333333333, total= 9.4min


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 19.0min remaining:    0.0s


[CV] lda_doc_topic_prior=0.01, lda_topic_word_prior=0.01 .............
[CV]  lda_doc_topic_prior=0.01, lda_topic_word_prior=0.01, score=54.50000000000001, total= 9.5min
[CV] lda_doc_topic_prior=0.01, lda_topic_word_prior=0.01 .............
[CV]  lda_doc_topic_prior=0.01, lda_topic_word_prior=0.01, score=58.83333333333333, total= 9.3min
[CV] lda_doc_topic_prior=0.0125, lda_topic_word_prior=0.01 ...........
[CV]  lda_doc_topic_prior=0.0125, lda_topic_word_prior=0.01, score=55.166666666666664, total= 9.3min
[CV] lda_doc_topic_prior=0.0125, lda_topic_word_prior=0.01 ...........
[CV]  lda_doc_topic_prior=0.0125, lda_topic_word_prior=0.01, score=57.33333333333333, total= 9.3min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 56.4min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=CustomLDA(clf_ntopics=3, clf_ntopwords=3, lda_doc_topic_prior=None,
     lda_n_components=100, lda_n_jobs=2, lda_random_state=0,
     lda_topic_word_prior=None, vect_code_max_df=0.8,
     vect_code_max_features=15000, vect_code_min_df=5,
     vect_code_ngram_range=(1, 2), vect_text_max_df=...df=0.8,
     vect_title_max_features=15000, vect_title_min_df=5,
     vect_title_ngram_range=(1, 2)),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'lda_doc_topic_prior': [0.008333333333333333, 0.01, 0.0125], 'lda_topic_word_prior': [0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=3)

In [26]:
clf.best_params_

{'lda_doc_topic_prior': 0.008333333333333333, 'lda_topic_word_prior': 0.01}

In [27]:
clf.best_score_

57.416617745298488

In [150]:
dataset.head(10)

,TText,TText_NEG,PText,TCode,PCode,TTitle,TTitle_NEG,Title,PTags,Tags
Id,,,,,,,,,,
48096039,set send cooki via ajax store browser ask logi...,set send cooki via ajax store browser ask_NEG ...,I can set and send cookie via ajax which is s...,None,None,set send cooki via ajax,set send cooki via ajax,Set and Send cookie via ajax,"[ajax, curl, cookies]",<ajax><curl><cookies>
48089460,use api sync tabl data need queri data need ca...,use api sync tabl data need queri data need ca...,For now I am using the api to sync the tables...,None,None,configur kylin synchron tabl period,configur kylin synchron tabl period,Can we configure Kylin to synchronize all the ...,[kylin],<kylin>
48091983,'m use codeignit php framework ion_auth authen...,'m use codeignit php framework ion_auth authen...,I'm using Codeigniter as my PHP framework and...,None,None,implement share databas multi tenant applic php,implement share databas multi tenant applic php,Implementing shared database for multi tenant ...,"[php, mysql, codeigniter, saas]",<php><mysql><codeigniter><saas>
48077316,tri run code octav written basic matlab one fu...,tri run code octav written basic matlab one fu...,I am trying to run a code in Octave written b...,error :' sylvester ' undefined near line column,error: 'sylvester' undefined near line 14 colu...,sylvest function octav,sylvest function octav,Sylvester function in Octave,"[matlab, octave]",<matlab><octave>
48092975,use laravel 's artisan command creat databas t...,use laravel 's artisan command creat databas t...,I am using laravel's artisan command to creat...,php artisan migrate table_1 table_1_table_1_si...,php artisan migrate table_1 table_1_table_1_si...,laravel migrat error creat tabl foreign key,laravel migrat error creat tabl foreign key,Laravel migration error on creating table with...,"[php, mysql, laravel]",<php><mysql><laravel>
48144183,'m tri find id object insid array object _id f...,'m tri find id object insid array object _id f...,I'm trying to find the id of an object inside...,var CardSchema = new mongoose . Schema ({ beNa...,var CardSchema = new mongoose.Schema({\n beNa...,find _id array object mongodb databas,find _id array object mongodb databas,Find by _id on an array of objects in mongodb ...,"[mongodb, mongoose, mongoose-schema]",<mongodb><mongoose><mongoose-schema>
48186237,use angular want bind complet json object sele...,use angular want bind complet json object sele...,I am using with in Angular 5. I want to bin...,"< select class ="" w3 - input dropDown form - c...","<select class=""w3-input dropDown form-control""...",two way data bind work select,two way data bind work_NEG select_NEG,Two way data binding not working for <select>,"[typescript, angular5]",<typescript><angular5>
48051220,make 3d game player rotat view point via mous ...,make 3d game player rotat view point via mous ...,I am making a 3D game in which the player can...,private static void changeRotation () { angle ...,private static void changeRotation(){\n ang...,3d rotat ratio via mous placement screen java,3d rotat ratio via mous placement screen java,3D rotation ratios via mouse placement on scre...,"[java, rotation, angle]",<java><3d><rotation><angle>
48168060,current tri write xml file use path `` system....,current tri write xml file use path `` system....,"I am currently trying to write an XML file, w...","< bean class ="" org . springframework . beans ...","<bean class=""org.springframework.beans.factory...",spring 2.0 xml propertyplaceholderconfigur sys...,spring 2.0 xml propertyplaceholderconfigur sys...,Spring 2.0 XML PropertyPlaceholderConfigurer S...,"[java, xml, spring, properties, system]",<java><xml><spring><properties><system>


In [17]:
article = [48168060, 48150752]
samplepost = dataset[['TText', 'TTitle', 'TCode']].loc[article]
print(dataset["PTags"].loc[article])
np.array(clf.predict(samplepost))

Id
48168060    [java, xml, spring, properties, system]
48150752             [android, xml, android-studio]
Name: PTags, dtype: object


array([[['name', 'xml', 'http'],
        ['script', 'file', 'run'],
        ['org', 'springframework', 'org springframework']],

       [['android', 'id', 'layout_height'],
        ['the', 'to', 'is'],
        ['item', 'menu', 'items']]],
      dtype='<U19')

In [197]:
dataset[['TText', 'TTitle', 'TCode', 'PTags']]

,TText,TTitle,TCode,PTags
Id,,,,
48053160,look around n't found someth help arduino uno ...,ca n't get data mysql databas arduino variabl,"<? php $ db_name ="" aquarium ""; $ user ="" root...","[php, mysql, http, get, arduino]"
48164249,environ python 3.6 ananconda 5.0.1,ca n't download newsgroup data via python code,None,[scikit-learn]
48116439,'s scenario got two entiti three type modul ar...,unabl rest put oper onetomani relat spring boot,@ Entity @ Data class Project {@ NotEmpty priv...,"[spring, spring-data-rest, spring-restcontroll..."
48166535,whmcs version drop contain list client get cli...,databas capsul manag work,jQuery ( document ). ready ( function ($ ){ $(...,"[ajax, whmcs]"
48143597,question regard memori access applic run compu...,access memori app without run admin privileg,None,"[memory, memory-management, access, memory-acc..."
48185162,given pseudo data -train data implement random...,groupbi model random forest algorithm get sing...,"rf = randomForest ( Default ~. , data = traind...","[r, machine-learning, random-forest, data-scie..."
48170117,vba experi hope way without use macro program ...,record api ticker data excel tabl,None,"[excel, api, store, record, ticker]"
48178451,'m work exercis 14.2-4 clrs intro algorithm 3e...,asymptot run time print key red-black tree fal...,"RB - ENUMERATE ( x , a , b ) T = red - black t...","[algorithm, time-complexity, binary-tree, bina..."
48077459,code json data arraylist 'm take one parent he...,pass one json two differ data expand listview,timez _listDataHeader _listdataChild LinearLay...,"[android, expandablelistview]"


# NMF version

In [158]:

class CustomNMF(BaseEstimator, TransformerMixin):

    def __init__(self, **params):        
        self.vect_text_params = {'ngram_range':(1,1), 'max_df':1., 'min_df':1, 'max_features':None}
        self.vect_title_params = {'ngram_range':(1,1), 'max_df':1., 'min_df':1, 'max_features':None}
        self.vect_code_params = {'ngram_range':(1,1), 'max_df':1., 'min_df':1, 'max_features':None}
        
        self.nmf_params = {'n_components':10, 'random_state':0, 'init':'nndsvd', 'alpha':0, 'l1_ratio':0}
        
        self.clf_params = {'ntopwords':5, 'ntopics':5}
        
        self.params = {**{'vect_text_'+k:v for k,v in self.vect_text_params.items()},
                       **{'vect_title_'+k:v for k,v in self.vect_title_params.items()},
                       **{'vect_code_'+k:v for k,v in self.vect_code_params.items()},
                       **{'nmf_'+k:v for k,v in self.nmf_params.items()},
                       **{'clf_'+k:v for k,v in self.clf_params.items()},
                       **params}
        
        self.update_params()

    def predict(self, X, y=None):
        ntopwords = self.clf_params['ntopwords']
        ntopics = self.clf_params['ntopics']
        W = self.transform(X)
        components = self.components_()
        feature_names = self.get_feature_names()

        toreturn = []
        
        for document in W:
            #select the most important topics
            docprediction = []
            for topic in document.argsort()[:-ntopics-1:-1]:
                tags = [feature_names[i] for i in components[topic].argsort()[:-ntopwords-1:-1]]
                docprediction.append(tags)
            toreturn.append(docprediction)
        return toreturn
    
    def fit(self, X, y=None):
        dftext = X['TText']
        dftitle = X['TTitle']
        dfcode = X['TCode']
        #Text preparation
        self.textcvect = TfidfVectorizer(tokenizer=None, vocabulary=None, **self.vect_text_params)
        text = self.textcvect.fit_transform(dftext)
        
        #Title preparation
        self.titlecvect = TfidfVectorizer(tokenizer=None, vocabulary=None, **self.vect_title_params)
        title = self.titlecvect.fit_transform(dftitle)
        
        #Code preparation
        self.codecvect = TfidfVectorizer(tokenizer=None, vocabulary=None, **self.vect_code_params)
        code = self.codecvect.fit_transform(dfcode)
        
        #LDA preparation
        self.nmf = NMF(**self.nmf_params)
        complete = scipy.sparse.hstack((text, title, code))
        self.nmf.fit(complete)
        
        return self
    
    def transform(self, X, y=None):
        dftext = X['TText']
        dftitle = X['TTitle']
        dfcode = X['TCode']
        
        #Text preparation
        textvect = self.textcvect.transform(dftext)
        
        #Title preparation
        titlevect = self.titlecvect.transform(dftitle)
    
        #Code preparation
        codevect = self.codecvect.transform(dfcode)
        
        nmf = self.nmf.transform(scipy.sparse.hstack((textvect, titlevect, codevect)))
        
        return nmf
    
    def get_feature_names(self):
        textfnames = self.textcvect.get_feature_names()
        titlefnames = self.titlecvect.get_feature_names()
        codefnames = self.codecvect.get_feature_names()
        toreturn = []
        toreturn.extend(textfnames)
        toreturn.extend(titlefnames)
        toreturn.extend(codefnames)
        
        return toreturn
    
    def components_(self):        
        return self.nmf.components_
    
    def get_params(self, deep=True):
        return self.params
    
    def set_params(self, **params):
        self.params = {**self.params, **params}
        self.update_params()
        return self
        
    def update_params(self):
        
        self.vect_text_params = {k[10:]:v for k,v in self.params.items() if k.startswith('vect_text_')}
        self.vect_title_params = {k[11:]:v for k,v in self.params.items() if k.startswith('vect_title_')}
        self.vect_code_params = {k[10:]:v for k,v in self.params.items() if k.startswith('vect_code_')}
        
        self.nmf_params = {k[4:]:v for k,v in self.params.items() if k.startswith('nmf_')}
        
        self.clf_params = {k[4:]:v for k,v in self.params.items() if k.startswith('clf_')}

        return self

    def score(self, X=None, y=None):
        numcomponent = self.nmf_params['n_components']
        self.nametopics()
        topicsscore = [1/len(x) if len(x) !=0 else 0 for x in self.topicsnamed.values()]
        return numcomponent * np.array(topicsscore).mean()
    
    def nametopics(self):
        self.topicsnamed = {}
        self.namedtopics = {}
        ntopwords = self.clf_params['ntopwords']
        ntopics = self.clf_params['ntopics']
        components = self.components_()
        feature_names = self.get_feature_names()
        
        for topicnum, topiccomposition in enumerate(components):
            tags = [feature_names[i] for i in topiccomposition.argsort()[:-ntopwords-1:-1]]
            self.topicsnamed[topicnum] = [tag for tag in tags if tag in existintags]
            for tag in tags:
                if tag in existintags:
                    if tag in self.namedtopics.keys():
                        temp = list(self.namedtopics[tag])
                        temp.append(topicnum)
                        self.namedtopics[tag] = set(temp)
                    else:
                        self.namedtopics[tag] = set([topicnum])
        
        return self


customNMF = CustomNMF()


In [159]:
params = {'clf_ntopics': 3,
 'clf_ntopwords': 3,
 'nmf_n_components': 100,
 'nmf_random_state': 0,
 'vect_code_max_df': 0.8,
 'vect_code_max_features': 15000,
 'vect_code_min_df': 5,
 'vect_code_ngram_range': (1, 2),
 'vect_text_max_df': 0.8,
 'vect_text_max_features': 15000,
 'vect_text_min_df': 10,
 'vect_text_ngram_range': (1, 3),
 'vect_title_max_df': 0.8,
 'vect_title_max_features': 15000,
 'vect_title_min_df': 5,
 'vect_title_ngram_range': (1, 2)}

customNMF = CustomNMF(**params)
customNMF.fit(dataset[['TText', 'TTitle', 'TCode']])

CustomNMF(clf_ntopics=3, clf_ntopwords=3, nmf_alpha=0, nmf_init='nndsvd',
     nmf_l1_ratio=0, nmf_n_components=100, nmf_random_state=0,
     vect_code_max_df=0.8, vect_code_max_features=15000,
     vect_code_min_df=5, vect_code_ngram_range=(1, 2),
     vect_text_max_df=0.8, vect_text_max_features=15000,
     vect_text_min_df=10, vect_text_ngram_range=(1, 3),
     vect_title_max_df=0.8, vect_title_max_features=15000,
     vect_title_min_df=5, vect_title_ngram_range=(1, 2))

In [164]:
customNMF.score()
#customNMF.namedtopics

23.333333333333332

In [168]:
article = [48168060, 48150752]
samplepost = dataset[['TText', 'TTitle', 'TCode']].loc[article]
print(dataset["PTags"].loc[article])
np.array(customNMF.predict(samplepost))

Id
48168060    [java, xml, spring, properties, system]
48150752             [android, xml, android-studio]
Name: PTags, dtype: object


array([[['spring', 'spring', 'boot', 'spring boot', 'boot'],
        ['user', 'user', 'user', 'login', 'authent'],
        ['field', 'field', 'field', 'input field', 'xml']],

       [['android', 'android', 'devic', 'android studio', 'devic'],
        ['android', 'layout', 'xml', 'layout', 'layout_width'],
        ['visual', 'studio', 'visual studio', 'visual', 'studio']]],
      dtype='<U14')

In [167]:
customNMF.set_params(clf_ntopwords=5)

CustomNMF(clf_ntopics=3, clf_ntopwords=5, nmf_alpha=0, nmf_init='nndsvd',
     nmf_l1_ratio=0, nmf_n_components=100, nmf_random_state=0,
     vect_code_max_df=0.8, vect_code_max_features=15000,
     vect_code_min_df=5, vect_code_ngram_range=(1, 2),
     vect_text_max_df=0.8, vect_text_max_features=15000,
     vect_text_min_df=10, vect_text_ngram_range=(1, 3),
     vect_title_max_df=0.8, vect_title_max_features=15000,
     vect_title_min_df=5, vect_title_ngram_range=(1, 2))